In [1]:
# This is the tutorial found at https://gist.github.com/tkosciol/29de5198a4be81559a075756c2490fde
# Setup

import qiime2
from tempfile import mkdtemp
from qiime2.plugins import demux, deblur, quality_filter, metadata, feature_table, alignment, phylogeny, diversity, emperor, feature_classifier, taxa, composition

# Create directories for
# the project
!mkdir -p qiime2-moving-pictures-tutorial
# the data
!mkdir -p $workdir/emp-single-end-sequences

# Set working directory
workdir="/mnt/d/dev/pda/notebook/qiime2-moving-pictures-tutorial"
%cd $workdir

# Get Sample metadata
!wget -nc -O $workdir/"sample-metadata.tsv" "https://data.qiime2.org/2020.11/tutorials/moving-pictures/sample_metadata.tsv"
# Barcodes data
!wget -nc -O $workdir/"emp-single-end-sequences/barcodes.fastq.gz" "https://data.qiime2.org/2020.11/tutorials/moving-pictures/emp-single-end-sequences/barcodes.fastq.gz"
# Sequence data
!wget -nc -O $workdir/"emp-single-end-sequences/sequences.fastq.gz" "https://data.qiime2.org/2020.11/tutorials/moving-pictures/emp-single-end-sequences/sequences.fastq.gz"
# Taxonomic data
!wget -nc -O $workdir"/gg-13-8-99-515-806-nb-classifier.qza" "https://data.qiime2.org/2020.11/common/gg-13-8-99-515-806-nb-classifier.qza"

/mnt/d/dev/pda/notebook/qiime2-moving-pictures-tutorial
File ‘/mnt/d/dev/pda/notebook/qiime2-moving-pictures-tutorial/sample-metadata.tsv’ already there; not retrieving.
File ‘/mnt/d/dev/pda/notebook/qiime2-moving-pictures-tutorial/emp-single-end-sequences/barcodes.fastq.gz’ already there; not retrieving.
File ‘/mnt/d/dev/pda/notebook/qiime2-moving-pictures-tutorial/emp-single-end-sequences/sequences.fastq.gz’ already there; not retrieving.
File ‘/mnt/d/dev/pda/notebook/qiime2-moving-pictures-tutorial/gg-13-8-99-515-806-nb-classifier.qza’ already there; not retrieving.


In [2]:
# Loading data

single_end_sequences = qiime2.Artifact.import_data('EMPSingleEndSequences', workdir+'/emp-single-end-sequences')
sample_metadata = qiime2.Metadata.load(workdir+'/sample-metadata.tsv')

In [3]:
# Demultiplexing

demux_sequences = demux.methods.emp_single(single_end_sequences, sample_metadata.get_column('barcode-sequence'))
demux_summary = demux.visualizers.summarize(demux_sequences.per_sample_sequences)
demux_summary.visualization

<visualization: Visualization uuid: 69712095-e72d-448f-81cb-e2ac63389c9f>

In [4]:
# Deblur

demux_filter_stats = quality_filter.methods.q_score(demux_sequences.per_sample_sequences)

deblur_sequences = deblur.methods.denoise_16S(demux_sequences.per_sample_sequences, trim_length=120, sample_stats=True)
filter_stats = metadata.visualizers.tabulate(demux_filter_stats.filter_stats.view(qiime2.Metadata))
filter_stats.visualization

/root/anaconda3/envs/qiime/lib/python3.6/site-packages/q2_quality_filter/_filter.py:89: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  phred_offset = yaml.load(metadata_view)['phred-offset']


<visualization: Visualization uuid: 64c8428c-7657-4db8-8a32-14ccdbe1d7cc>

In [5]:
deblur_stats = deblur.visualizers.visualize_stats(deblur_sequences.stats)
deblur_stats.visualization

<visualization: Visualization uuid: 1462187d-2738-4f5f-bc5f-556ebf610ddb>

In [6]:
# Feature table

output_viz = feature_table.visualizers.summarize(deblur_sequences.table)
output_viz.visualization

/root/anaconda3/envs/qiime/lib/python3.6/site-packages/q2_feature_table/_summarize/_visualizer.py:122: FutureWarning: The signature of `Series.to_csv` was aligned to that of `DataFrame.to_csv`, and argument 'header' will change its default value from False to True: please pass an explicit value to suppress this warning.
  os.path.join(output_dir, 'sample-frequency-detail.csv'))
/root/anaconda3/envs/qiime/lib/python3.6/site-packages/q2_feature_table/_summarize/_visualizer.py:124: FutureWarning: The signature of `Series.to_csv` was aligned to that of `DataFrame.to_csv`, and argument 'header' will change its default value from False to True: please pass an explicit value to suppress this warning.
  os.path.join(output_dir, 'feature-frequency-detail.csv'))


<visualization: Visualization uuid: 37cb76e9-9305-4a07-a9ae-9e0cf29cd799>

In [7]:
deblur_feature_table_summary = feature_table.visualizers.tabulate_seqs(deblur_sequences.representative_sequences)
deblur_feature_table_summary.visualization

<visualization: Visualization uuid: 71549186-7655-4567-8303-d15ffa774bba>

In [8]:
# Alignment

mafft_alignment = alignment.methods.mafft(deblur_sequences.representative_sequences)
masked_mafft_alignment = alignment.methods.mask(mafft_alignment.alignment)
unrooted_tree = phylogeny.methods.fasttree(masked_mafft_alignment.masked_alignment)
rooted_tree = phylogeny.methods.midpoint_root(unrooted_tree.tree)

Running external command line application. This may print messages to stdout and/or stderr.
The command being run is below. This command cannot be manually re-run as it will depend on temporary files that no longer exist.

Command: mafft --preservecase --inputorder --thread 1 /tmp/qiime2-archive-l8ea5l_8/37673a48-945c-488d-bb54-0824179ec288/data/dna-sequences.fasta

Running external command line application. This may print messages to stdout and/or stderr.
The command being run is below. This command cannot be manually re-run as it will depend on temporary files that no longer exist.

Command: FastTree -quote -nt /tmp/qiime2-archive-sw1br9nx/e8592627-1397-430a-be99-c8bd1fe66eb5/data/aligned-dna-sequences.fasta



In [9]:
# Alpha and beta diversity

core_metrics = diversity.pipelines.core_metrics_phylogenetic(table = deblur_sequences.table, phylogeny = rooted_tree.rooted_tree, sampling_depth = 1109, metadata = sample_metadata)

/root/anaconda3/envs/qiime/lib/python3.6/site-packages/sklearn/metrics/pairwise.py:1761: DataConversionWarning: Data was converted to boolean for metric jaccard
  warnings.warn(msg, DataConversionWarning)
/root/anaconda3/envs/qiime/lib/python3.6/site-packages/skbio/stats/ordination/_principal_coordinate_analysis.py:152: RuntimeWarning: The result contains negative eigenvalues. Please compare their magnitude with the magnitude of some of the largest positive eigenvalues. If the negative ones are smaller, it's probably safe to ignore them, but if they are large in magnitude, the results won't be useful. See the Notes section for more details. The smallest eigenvalue is -0.016715512766180045 and the largest is 3.612148448868697.
  RuntimeWarning
/root/anaconda3/envs/qiime/lib/python3.6/site-packages/skbio/stats/ordination/_principal_coordinate_analysis.py:152: RuntimeWarning: The result contains negative eigenvalues. Please compare their magnitude with the magnitude of some of the largest

In [10]:
# Alpha group significance

faith_pd_group_significance = diversity.actions.alpha_group_significance(core_metrics.faith_pd_vector, sample_metadata)
evenness_group_significance = diversity.actions.alpha_group_significance(core_metrics.evenness_vector, sample_metadata)

In [11]:
faith_pd_group_significance.visualization

<visualization: Visualization uuid: 235f6685-ae93-4d95-8e4c-0a5cd9aeb99b>

In [12]:
evenness_group_significance.visualization

<visualization: Visualization uuid: 745e9a09-062d-4eda-95a0-d1db911fabb3>

In [13]:
# Beta group significance

uUniFrac_BodySite_significance = diversity.actions.beta_group_significance(core_metrics.unweighted_unifrac_distance_matrix, sample_metadata.get_column('body-site'))
uUniFrac_Subject_significance = diversity.actions.beta_group_significance(core_metrics.unweighted_unifrac_distance_matrix, sample_metadata.get_column('subject'))


/root/anaconda3/envs/qiime/lib/python3.6/site-packages/seaborn/categorical.py:403: UserWarning: Attempted to set non-positive left xlim on a log-scaled axis.
Invalid limit will be ignored.
  ax.set_xlim(-.5, len(self.plot_data) - .5, auto=None)


In [14]:
uUniFrac_BodySite_significance.visualization

<visualization: Visualization uuid: 8ab664db-fbbd-4e9b-a1ae-fcee91d74749>

In [15]:
uUniFrac_Subject_significance.visualization

<visualization: Visualization uuid: f664cb2e-c654-4338-8c63-ce90de623701>

In [16]:
# Emperor PCoA

emperor_plot = emperor.visualizers.plot(core_metrics.unweighted_unifrac_pcoa_results, sample_metadata, custom_axes=['days-since-experiment-start'])

In [17]:
emperor_plot.visualization

<visualization: Visualization uuid: 43dd2a88-b516-4cdd-9fbe-d7fe6adba1ef>

In [18]:
# Alpha rarefaction plotting

rarefaction = diversity.actions.alpha_rarefaction(table = deblur_sequences.table, max_depth = 4000, phylogeny = rooted_tree.rooted_tree, metadata = sample_metadata)

In [19]:
rarefaction.visualization

<visualization: Visualization uuid: e4ce2aa7-9c4b-4b84-ac83-1e375588e5b5>

In [20]:
# Taxonomic analysis

!qiime tools export --input-path $workdir/gg-13-8-99-515-806-nb-classifier.qza --output-path $workdir/classifier

Exported /mnt/d/dev/pda/notebook/qiime2-moving-pictures-tutorial/gg-13-8-99-515-806-nb-classifier.qza as TaxonomicClassiferTemporaryPickleDirFmt to directory /mnt/d/dev/pda/notebook/qiime2-moving-pictures-tutorial/classifier


In [21]:
gg_classifier = qiime2.Artifact.import_data('TaxonomicClassifier', workdir+'/classifier/')
taxonomy = feature_classifier.methods.classify_sklearn(reads = deblur_sequences.representative_sequences, classifier = gg_classifier)
taxonomy_classification = metadata.visualizers.tabulate(taxonomy.classification.view(qiime2.Metadata))
taxonomy_classification.visualization

<visualization: Visualization uuid: 0fe5590a-c0f0-40ec-ab7f-c66342581596>

In [22]:
taxa_bar_plot = taxa.visualizers.barplot(deblur_sequences.table, taxonomy.classification, sample_metadata)
taxa_bar_plot.visualization

<visualization: Visualization uuid: 46038fad-99b1-4761-9e01-fefeac504f00>

In [23]:
# Differential abundance testing with ANCOM

gut_deblur = feature_table.methods.filter_samples(deblur_sequences.table, metadata = sample_metadata, where = "[body-site]='gut'")
gut_deblur_composition = composition.actions.add_pseudocount(gut_deblur.filtered_table)

ancom_gut_deblur = composition.actions.ancom(table = gut_deblur_composition.composition_table, metadata = sample_metadata.get_column('subject'))
ancom_gut_deblur.visualization

/root/anaconda3/envs/qiime/lib/python3.6/site-packages/scipy/stats/stats.py:3641: F_onewayConstantInputWarning: Each of the input arrays is constant;the F statistic is not defined or infinite
  warnings.warn(F_onewayConstantInputWarning())


<visualization: Visualization uuid: e2e918c0-4819-41a0-ba2f-d2e78a874746>

In [24]:
gut_table_l6 = taxa.methods.collapse(table = gut_deblur.filtered_table, taxonomy = taxonomy.classification, level = 6)
gut_table_l6_composition = composition.actions.add_pseudocount(gut_table_l6.collapsed_table)
ancom_gut_table_l6 = composition.actions.ancom(table = gut_table_l6_composition.composition_table, metadata = sample_metadata.get_column('subject'))

In [25]:
ancom_gut_table_l6.visualization

<visualization: Visualization uuid: 06a434e4-d18f-4322-a403-f32211819757>